In [ ]:
# Python Script
# To download a subet (1000 images + json) of the Amazon dataset

In [ ]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.7 MB/s eta 0:00:00


In [ ]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import os

# Use unsigned config (no AWS credentials needed)
s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED), region_name="us-east-1")

bucket = "aft-vbi-pds"
img_prefix = "bin-images/"
meta_prefix = "metadata/"

# Make local folders
os.makedirs("bin-images", exist_ok=True)
os.makedirs("metadata", exist_ok=True)

# List first 1000 images
response = s3.list_objects_v2(Bucket=bucket, Prefix=img_prefix, MaxKeys=1200)  # ask for a bit more than 1000 to skip bad keys
count = 0

for obj in response.get("Contents", []):
    key = obj["Key"]
    filename = key.split("/")[-1]

    # Only process .jpg files
    if not filename.endswith(".jpg"):
        continue

    # Download image
    s3.download_file(bucket, key, f"bin-images/{filename}")

    # Download corresponding metadata
    json_key = meta_prefix + filename.replace(".jpg", ".json")
    s3.download_file(bucket, json_key, f"metadata/{filename.replace('.jpg', '.json')}")

    count += 1
    if count >= 1000:  # stop after 1000
        break

print(f"✅ Download complete: {count} images + metadata")